<a href="https://colab.research.google.com/github/pranitha-s/Music-Recommendation-System-using-Python/blob/main/Spotify_Music_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spotipy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 6.5 MB/s eta 0:00:00


In [2]:
import requests
import base64

CLIENT_ID = '91f761611636403088e633a7d177f6e0'
CLIENT_SECRET = 'da0feb9ce0404e79a3d68a726d34d051'

client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [3]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):

    sp = spotipy.Spotify(auth=access_token)


    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')


    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']


        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None


        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None


        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None


        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,

        }

        music_data.append(track_data)


    df = pd.DataFrame(music_data)

    return df

In [4]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'


music_df = get_trending_playlist_data(playlist_id, access_token)

print(music_df)

                       Track Name                      Artists  \
0                        fukumean                        Gunna   
1                           Prada  cassö, RAYE, D-Block Europe   
2                 I'm Good (Blue)     David Guetta, Bebe Rexha   
3      FE!N (feat. Playboi Carti)  Travis Scott, Playboi Carti   
4   Jimmy Cooks (feat. 21 Savage)             Drake, 21 Savage   
..                            ...                          ...   
95    Another Love - Tiësto Remix            Tom Odell, Tiësto   
96    pushin P (feat. Young Thug)    Gunna, Future, Young Thug   
97          Rainfall (Praise You)                    Tom Santa   
98   Kernkraft 400 (A Better Day)                   Topic, A7S   
99                    Motley Crew                  Post Malone   

                      Album Name                Album ID  \
0               a Gift & a Curse  5qmZefgh78fN3jsyPPlvuw   
1                          Prada  5MU0RmBSpoSxOPYBfcobDc   
2                I'm Good (

In [5]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [7]:
def calculate_weighted_popularity(release_date):
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    time_span = datetime.now() - release_date

    weight = 1 / (time_span.days + 1)
    return weight

In [8]:
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [9]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [12]:
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    hybrid_recommendations = content_based_rec
    new_row = {
    'Track Name': input_song_name,
    'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
    'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
    'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
    'Popularity': weighted_popularity_score
    }
    hybrid_recommendations = pd.concat([hybrid_recommendations, pd.DataFrame([new_row])], ignore_index=True)


    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [13]:
input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'I'm Good (Blue)':
                   Track Name                                     Artists  \
3  FE!N (feat. Playboi Carti)                 Travis Scott, Playboi Carti   
4                Call It Love                     Felix Jaehn, Ray Dalton   
1                       REACT  Switch Disco, Ella Henderson, Robert Miles   
0                        BOTH                      Tiësto, 21 Savage, BIA   
2               Where You Are                          John Summit, Hayla   

      Album Name Release Date  Popularity  
3         UTOPIA   2023-07-28        91.0  
4   Call It Love   2022-09-16        81.0  
1          REACT   2023-01-13        80.0  
0           BOTH   2023-08-29        79.0  
2  Where You Are   2023-03-03        77.0  
